# T5 Reranking with A2A
This notebook shows A2A conveniently allows you to download datasets, train a reranker and score it. You only need set the A2A API up once, then you can run any IR task on any device you wish, without need for any external resources on device.

The Notebook demonstrates the following tasks
1. Downloading the training dataset from A2A
2. Training a T5 reranker
3. Initial retrieval with BM25 through A2A
4. Reranking the results of BM25
5. Scoring the reranking using A2A


## Imports

In [ ]:
%pip install --quiet sentencepiece pytorch_lightning transformers

In [ ]:
import os
import zipfile
import json
from io import BytesIO
import warnings
import requests
import argparse
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
import sentencepiece
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# set seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

## Data cleaning and Model Training Functions

In [ ]:
def unzip_and_read_json(response, output_folder="qrel_docs"):
    """Unzip the qrel_docs response and return the json"""
    # extract the zip file contents
    zipped = zipfile.ZipFile(BytesIO(response.content))
    zipped.extractall(output_folder)

    # read the json from zip file
    root, dirs, files = next(os.walk("qrel_docs")) # the unzipped folder contains a single json file, find its name
    file_path = os.path.join(root, files[0])
    with open(file_path, 'r') as file:
        json_dict = json.loads(file.read())
    
    # remove file and folder
    os.remove(file_path)
    os.rmdir(output_folder)
    return json_dict

In [ ]:
# This cell contains classes and functions for data cleaning and model training
def prepare_data_and_train_reranker(
        topics, docs, qrels,
        max_epochs=5,
        batch_size=2,
        final_batch_size=32, # Target batch size after gradient accumulation
        max_input_size=1024, # Input size of the T5 reranker
        t5path="castorini/monot5-base-med-msmarco", # from Nogueira et al. 2020 https://aclanthology.org/2020.findings-emnlp.63/
        irrelevance_threshold = 0, # labels >= 0 are considered positive examples
        label_words = ["false", "true"]
    ):
    topics, docs, qrels = transform_training_data(topics, docs, qrels)
    
    # Initialize the dataset
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    label_indexes = torch.tensor(tokenizer(label_words)["input_ids"]).long()[:, 0] # token index of each label word
    datamodule = IRDataModule(topics, docs, qrels, batch_size=batch_size, labels=label_words, 
                              irrelevance_threshold=irrelevance_threshold, max_input_size=max_input_size)

    # Initialize model
    model = T5Reranker(label_indexes, t5path=t5path)

    # Train with pytorch lightning
    trainer = pl.Trainer(
        logger=True, enable_progress_bar=True, 
        val_check_interval=128, 
        limit_val_batches=32, 
        gpus=1,
        max_epochs=max_epochs, 
        accumulate_grad_batches=int(final_batch_size/batch_size)
    )
    trainer.fit(model, datamodule)
    return model

def get_df(list_of_dicts, relevant_cols):
    """Convert the topic and document dictionaries into a pandas dataframe"""
    rows = [[d[col] for col in relevant_cols if col in d] for d in list_of_dicts] # form 2 dimensional array
    df = pd.DataFrame(rows, columns=relevant_cols)
    df = df.dropna(axis=0) # drop rows that have na values (e.g. in sigir_ct 9 trials lack inclusion criteria)
    return df

def transform_training_data(topics, docs, qrels):
    """Convert topics, documents and qrels into dataframes"""
    topics_df = get_df(topics, ["id", "description"]).rename({"id": "topic_id"}, axis=1)
    topics_df["topic_id"] = topics_df["topic_id"].astype(int)
    docs_df = pd.DataFrame(docs).T
    docs_df = docs_df[["id", "brief_title", "inclusion"]].rename(columns={"id": "doc_id"})
    qrels_df = pd.DataFrame(qrels, columns=["topic_id", "blank", "doc_id", "relevance_score"]).drop("blank", axis=1)
    return qrels_df, docs_df, topics_df

# Define dataset, datamodule and model classes for pytorch lightning training
class IRDataset(torch.utils.data.Dataset):
    def __init__(self, qrels, docs, topics, tokenizer_path="t5-base", labels=["irrelevant", "relevant"], irrelevance_threshold=0, max_input_size=512):
        """
        Load IR task data (from sigir_ct) as dataframes that represent qrels, docs and topics
        Labels: ordered list of labels, such that label[i] represents the label `i` in qrels
        Returns tokenized and concatenated topic and document in the form of: "Query: <query> Document: <doc>", also the tokenized mask
        """
        # Create an organised table containing queries and their relevant documents
        self.data = qrels.merge(topics, how="inner", on=["topic_id"])
        self.data = self.data.merge(docs, how="inner", on=["doc_id"])

        # Define tokenizer
        self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
        self.max_input_size = max_input_size
        
        # Tokenize the labels that T5 must predict
        self.irrelevance_threshold = irrelevance_threshold
        self.labels = [self.tokenizer(label, return_tensors='pt', padding='max_length', truncation=True, max_length=8) for label in labels]

    # tokenize the document and query, convert to CUDA
    def __getitem__(self, index):
        row = self.data.iloc[index]
        
        # Tokenize data
        query = row["description"]
        concatenated = f"Query: {query} Document: title: {row['brief_title']} criteria: {row['inclusion']}"
        outputs = self.tokenizer(concatenated, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_input_size)
        
        # Get correct label word, treating "partially relevant" as irrelevant (ideally the top 10 retreived items should be fully relevant, so train the model to identify this)
        label = 0 if row["relevance_score"] <= self.irrelevance_threshold else 1
        label = self.labels[label]
        return outputs["input_ids"], outputs["attention_mask"], label["input_ids"], label["attention_mask"]

    def __len__(self):
        return self.data.shape[0]

class IRDataModule(pl.LightningDataModule):
    """Data module that downloads IR training data, creates a train val split"""
    def __init__(self, topics, docs, qrels, batch_size=8, val_size=0.2, tokenizer_path="t5-base", 
            labels=["irrelevant", "relevant"], irrelevance_threshold=0, max_input_size=512):
        super().__init__()
        self.batch_size = batch_size
        self.val_size = val_size
        self.tokenizer_path = tokenizer_path
        self.labels = labels
        self.irrelevance_threshold = irrelevance_threshold
        self.max_input_size = max_input_size

        self.topics = topics
        self.docs = docs
        self._qrels = qrels

    def setup(self, stage=None):
        """create train val split"""
        train_qrels, val_qrels = train_test_split(self._qrels, test_size=self.val_size)
        self.qrels = {"train": train_qrels, "val": val_qrels}

    def get_dataloader(self, split):
        """Create dataset and dataloader for a given split"""
        dataset = IRDataset(self.qrels[split], self.docs, self.topics, 
            tokenizer_path=self.tokenizer_path, max_input_size=self.max_input_size, labels=self.labels)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, num_workers=0, pin_memory=True, shuffle=split=="train")
        return dataloader

    # Define these hooks that pytorch lightning needs to get the dataloaders
    def train_dataloader(self):
        return self.get_dataloader("train")

    def val_dataloader(self):
        return self.get_dataloader("val")

    def test_dataloader(self):
        return self.get_dataloader("val")

class T5Reranker(pl.LightningModule):
    """A simple T5 reranker, see https://arxiv.org/abs/2003.06713"""
    def __init__(self, label_indexes, t5path="t5-base", lr=0.001, dropout=0.1):
        """
        Initialize the model
        label_indexes[i]: the index of the token that represents label of `i`
        """
        super().__init__()
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5path, dropout_rate=dropout)
        self.lr = lr
        self.label_indexes = label_indexes

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

    def get_inputs(self, batch):
        """
        For each input: remove the empty dimension (1) created by the dataloaders batching process
        This creates tensors of size: (batch_size, sequence_length)
        """
        return [tensor.reshape(tensor.shape[0], tensor.shape[-1]) if tensor is not None else None for tensor in batch]

    def training_step(self, batch, batch_idx):
        """Train model to generate the label tokens"""
        input_ids, input_mask, output_ids, output_mask = self.get_inputs(batch)
        loss = self.t5(input_ids=input_ids, attention_mask=input_mask, labels=output_ids, decoder_attention_mask=output_mask).loss 
        self.log("train_loss", loss)
        return loss

    def forward(self, batch):
        """ Calculate class logits as the probability of the first subword. Should only be used for inference"""
        # Calculate the first token logits
        with torch.no_grad():
            input_ids, input_mask, _, _ = self.get_inputs(batch)
            start_tokens = torch.tensor([[1]]*input_ids.shape[0]).type_as(input_ids) # beginning of sentence tokens, used for generation
            output = self.t5.forward(input_ids=input_ids, attention_mask=input_mask, decoder_input_ids=start_tokens)

        # Return the logits of the label words, and the prediction
        word_logits = output["logits"][:, 0, self.label_indexes] # get the logits for those three words
        preds_class = torch.argmax(word_logits, dim=1) # the predicted class: 0 or 1
        preds_token = self.label_indexes[preds_class].type_as(preds_class)   # the token associated with that class (needed for validation)
        return preds_class, preds_token, word_logits

    def validation_step(self, batch, batch_idx):
        """return the predictions for a validation batch"""
        _, preds_token, _ = self.forward(batch)
        true_class = batch[2][:, 0, 0] # the tokens representing the true class labels
        return preds_token, true_class

    def binarise(self, labels):
        """Convert labels to binary, with 1 representing the positive class `self.label_indexes[-1]`"""
        int_labels = labels.cpu().detach().numpy().astype(int)
        pos_label =  int(self.label_indexes[-1])
        binary_labels = (int_labels == pos_label).astype(int)
        return binary_labels

    def validation_epoch_end(self, outputs):
        """Score model based on f1 of its predictions made through generating label tokens"""
        pred_class, true_class = list(zip(*outputs)) # zip creates two lists: the first one containing the 1st element of the tuples in outputs, second one containing the 2nd
        pred_class = torch.cat(pred_class) # concatenate the class labels from separate batches
        true_class = torch.cat(true_class)
        f1 = f1_score(self.binarise(pred_class), self.binarise(true_class))
        print("Validation F1 score:", f1)
        return f1

In [ ]:
# Reranking functions
def rerank_and_save_trec_output(init_retrieval, topics, bm25docs, reranker, rerank_limit=100, model_weight=1, max_input_size=1024, batch_size=2):
    # Put data in dataframes
    docs_df = pd.DataFrame(bm25docs).T.rename(columns={"id": "doc_id"})
    topics_df = get_df(topics, ["id", "description"]).rename({"id": "topic_id"}, axis=1)
    topics_df["topic_id"] = topics_df["topic_id"].astype(int)
    
    # perform reranking
    rank_dfs = rerank(init_retrieval, topics_df, docs_df, reranker, rerank_limit=rerank_limit, model_weight=model_weight, max_input_size=max_input_size, batch_size=batch_size)
    
    # write results to file
    output_file = "reranker.results"
    dfs_to_trec(rank_dfs, output_file)
    return output_file

# To score documents retrieved by BM25 we need the text of those documents
# First we define a function to convert BM25 results into a dataframe
def api_results_to_dfs(api_dict):
    """convert results from the api "/bm25/{dataset}" call to topic dfs"""
    topic_dfs = []
    for key, value in api_dict["rankings"].items():
        df = pd.DataFrame(value, columns=["doc_id", "score", "ignore", "ignore", "ignore"]).drop(columns=["ignore"])
        df["topic_id"] = int(key)
        df["rank"] = np.arange(df.shape[0]) + 1 # scores are already sorted in descending order, so the rank simply goes from 1 to 1000
        topic_dfs.append(df.copy())
    return topic_dfs

# This function converts the dataframes into the TREC result format
def df_to_trec(df, topic_id=0, file_name=None, rank_col="rank", run_name="ct2021_test"):
    """
    convert a single dataframe to the trec results format for scoring. 
    if file_name is passed, write the results to file, otherwise return the dataframe
    """
    # Convert the ranking to a score, so other ranking columns can be used
    df = df.copy()
    trec_df = df[["doc_id", rank_col]].copy()
    assert not np.any(trec_df[rank_col] == 0), "ranks starting at 0 cause a division by zero error!"
    trec_df["score"] = 1/trec_df[rank_col]
    trec_df = trec_df.sort_values("score", ascending=False)
    trec_df = trec_df[:1000]

    # Add columns in the TREC Format
    trec_df["Q0"] = "Q0"
    trec_df["run_name"] = run_name
    if "topic_id" in df.columns:
        trec_df["topic_id"] = df["topic_id"]
    else:
        trec_df["topic_id"] = topic_id + 1
        warnings.warn(f"No `topic_id` column found in `df`, setting topic_id as {topic_id} which may not be correct")
    trec_df = trec_df.rename(columns={"doc_id": "document_id"})

    trec_df = trec_df.reset_index()
    trec_df = trec_df.reindex(columns=["topic_id", "Q0", "document_id", rank_col, "score", "run_name"]) # reorder the columns
    
    if file_name is not None:
        trec_df.to_csv(file_name, sep="\t", header=False, index=False)
    return trec_df

def dfs_to_trec(dfs, file_name, rank_col="rank", run_name="ct2021_test"):
    """Takes in a list of query dfs and converts them to the TREC results fromat for scoring, note the dfs must be in the same order as the topic id"""
    results = []
    for topic_id, df in enumerate(dfs):
        results.append(df_to_trec(df, topic_id=topic_id, rank_col=rank_col, run_name=run_name))
    
    results = pd.concat(results, axis=0)
    results.to_csv(file_name, sep="\t", header=False, index=False)

# A new dataset to represent documents without labels is necessary
class RerankDataset(torch.utils.data.Dataset):
    def __init__(self, topic, docs, tokenizer_path="t5-base", max_input_size=512):
        """
        Dataset for reranking of ct2021 that concatenates the topic and all 1000 documents
        Returns tokenized and concatenated topic and document in the form of: "Query: <query> Document: <doc>", also the tokenized mask
        """
        self.topic = topic
        self.docs = docs
        self.dummy = torch.tensor([0])

        # Define tokenizer
        self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
        self.max_input_size = max_input_size

    # tokenize the document and query, convert to CUDA
    def __getitem__(self, index):
        doc = self.docs.iloc[index]
        query = self.topic
        concatenated = f"Query: {query} Document: title: {doc['brief_title']} criteria: {doc['inclusion']}"
        outputs = self.tokenizer(concatenated, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_input_size)

        # return dummy items to be interchangeable with IRDataset in the model.forward() function
        return outputs["input_ids"], outputs["attention_mask"], self.dummy, self.dummy

    def __len__(self):
        return self.docs.shape[0]

# We need to find someway to give a topic_id to the downloaded documents
# Do this using the bm25_baseline['rankings']
def rerank(init_retrieval, topics, init_docs_df, model, rerank_limit=100, model_weight=1, max_input_size=1024, batch_size=2):
    """Use `model` to perform reranking of the initial documents for each topic"""
    trainer = pl.Trainer(gpus=1, enable_progress_bar=True)
    init_dfs = pd.concat(api_results_to_dfs(init_retrieval))
    init_dfs = init_dfs[["topic_id", "doc_id", "score", "rank"]].rename(columns={"score": "bm25_score"}).copy()

    topic_dfs = []
    for i, row in topics.iterrows():
        # Get ranking df for this topic id
        topic_id = int(row["topic_id"])
        init_df = init_dfs[init_dfs["topic_id"] == topic_id].copy()
        
        # Download documents
        ranks_df = pd.merge(init_df, init_docs_df, how="left", on="doc_id") # join Bm25 scores with the docs needed for reranking
        assert init_df.shape[0] == ranks_df.shape[0], "There are duplicate documents in init_docs_df"

        # rerank with model, first create a dataset, then pass all items to the model's forward function
        top_docs_df = ranks_df.iloc[:rerank_limit].copy() # the top documents, in the same order as ranks_df
        topic_dataset = RerankDataset(row["description"], top_docs_df, max_input_size=max_input_size)        
        topic_dataloader = torch.utils.data.DataLoader(topic_dataset, batch_size=batch_size*2, num_workers=0, pin_memory=True, shuffle=False)
        output_batches = trainer.predict(model, topic_dataloader)
        output_logits = torch.cat([batch[2] for batch in output_batches], dim=0)  # output comes in batches, join the logits from each batch together
        pos_class_scores = torch.nn.functional.softmax(output_logits, dim=1)[:, 1]

        # Then normalize and interpolate the scores
        t5_scores = pos_class_scores.cpu().detach().numpy().astype(np.float64)
        ranks_df["t5_score"] = np.concatenate([t5_scores, np.zeros(1000 - t5_scores.shape[0])]) # fill up to the 1000th rank with 0 scores so that we can add to df
        ranks_df["bm25_score"] = ranks_df["bm25_score"].astype(np.float64)
        ranks_df["bm25_score"] = ranks_df["bm25_score"]/np.max(ranks_df["bm25_score"])
        ranks_df["score"] = model_weight*ranks_df["t5_score"] + (1-model_weight)*ranks_df["bm25_score"]

        # Convert to topic_df format to match docT5query_analysis.py
        ranks_df = ranks_df.sort_values("score", axis=0, ascending=False) # resort by scores
        ranks_df["rank"] = np.arange(ranks_df.shape[0]) + 1 # add the rank column, must start from 1
        topic_dfs.append(ranks_df)
        print(f"Topic {i} Rankings:")
        print(ranks_df[["topic_id", "doc_id", "t5_score", "bm25_score", "score", "rank"]])
    return topic_dfs

## Reranker training
Get qrel data and document content for docs present in the qrels. Then train a reranker

In [ ]:
## Get qrel data and document content for docs present in the qrels
qrels = requests.get('https://a2a.csiro.au/api/qrels/sigir_ct').json()
topics = requests.get('https://a2a.csiro.au/api/topics/sigir_ct').json()
response = requests.get('https://a2a.csiro.au/api/qrel_docs/sigir_ct')
qrel_docs = unzip_and_read_json(response)

In [ ]:
reranker = prepare_data_and_train_reranker(topics, qrel_docs, qrels, max_epochs=5)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name | Type                       | Params
----------------------------------------------------
0 | t5   | T5ForConditionalGeneration | 222 M 
----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Validation F1 score: 0.0


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validation F1 score: 0.3888888888888889


Validating: 0it [00:00, ?it/s]

Validation F1 score: 0.375


Validating: 0it [00:00, ?it/s]

Validation F1 score: 0.46511627906976744


## Reranking Inference

In [ ]:
bm25_baseline = requests.get('https://a2a.csiro.au/api/bm25/ct2021').json()

## Save bm25 results to disk as the file `bm25.results`
dfs_to_trec(api_results_to_dfs(bm25_baseline), 'bm25.results')

## Get data: topics and the top k(=20) documents in the baseline ranking
rerank_limit = 20
files = {'file': open('bm25.results','rb')}
bm25_docs = requests.post(f'https://a2a.csiro.au/api/result_docs/ct2021?k={rerank_limit}', files=files).json()
topics = requests.get('https://a2a.csiro.au/api/topics/ct2021').json()

NameError: ignored

In [ ]:
## Now we rerank the documents
reranked_output_path = rerank_and_save_trec_output(bm25_baseline, topics, bm25_docs, reranker, rerank_limit=rerank_limit)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs



Download docs for topic: 0, proper id: 1
Reranking topic: 0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Topic: 0
     topic_id       doc_id  t5_score  bm25_score     score  rank
3           1  NCT00004259  0.337368    0.897493  0.337368     1
1           1  NCT00114140  0.320821    0.939719  0.320821     2
16          1  NCT00720564  0.306807    0.783133  0.306807     3
11          1  NCT00631137  0.304751    0.826234  0.304751     4
18          1  NCT00006024  0.254806    0.782657  0.254806     5
..        ...          ...       ...         ...       ...   ...
349         1  NCT00381797  0.000000    0.586267  0.000000   996
350         1  NCT04302259  0.000000    0.586051  0.000000   997
351         1  NCT02054013  0.000000    0.585945  0.000000   998
352         1  NCT01905228  0.000000    0.585865  0.000000   999
999         1  NCT02202031  0.000000    0.479504  0.000000  1000

[1000 rows x 6 columns]

Download docs for topic: 1, proper id: 2
Reranking topic: 1


...

Predicting: 0it [00:00, ?it/s]

Topic: 73
     topic_id       doc_id  t5_score  bm25_score     score  rank
18         74  NCT01476501  0.605254    0.645343  0.605254     1
8          74  NCT00781261  0.561967    0.668373  0.561967     2
0          74  NCT02802384  0.347715    1.000000  0.347715     3
10         74  NCT00051636  0.335335    0.657595  0.335335     4
4          74  NCT00747994  0.330156    0.690289  0.330156     5
..        ...          ...       ...         ...       ...   ...
349        74  NCT01623739  0.000000    0.513572  0.000000   996
350        74  NCT01732549  0.000000    0.513564  0.000000   997
351        74  NCT01808794  0.000000    0.513506  0.000000   998
352        74  NCT03960437  0.000000    0.513462  0.000000   999
999        74  NCT03068351  0.000000    0.468040  0.000000  1000

[1000 rows x 6 columns]

Download docs for topic: 74, proper id: 75
Reranking topic: 74


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Topic: 74
     topic_id       doc_id  t5_score  bm25_score     score  rank
9          75  NCT02125825  0.598846    0.860359  0.598846     1
19         75  NCT02655315  0.541396    0.821838  0.541396     2
1          75  NCT00032812  0.447488    0.931438  0.447488     3
13         75  NCT02304848  0.443217    0.841722  0.443217     4
8          75  NCT03253965  0.409185    0.864153  0.409185     5
..        ...          ...       ...         ...       ...   ...
349        75  NCT04739423  0.000000    0.593051  0.000000   996
350        75  NCT00517842  0.000000    0.592655  0.000000   997
351        75  NCT04652583  0.000000    0.592653  0.000000   998
352        75  NCT03546790  0.000000    0.591353  0.000000   999
999        75  NCT04505527  0.000000    0.488165  0.000000  1000

[1000 rows x 6 columns]


## Reranking Evaluation

In [ ]:
files = {'file': open(reranked_output_path,'rb')}
reranked_results_graded = requests.post('https://a2a.csiro.au/api/eval/ct2021', files=files).json()
files = {'file': open(reranked_output_path,'rb')}
reranked_results_binary = requests.post('https://a2a.csiro.au/api/eval/ct2021_eligible', files=files).json()
print("Reranking Results:", reranked_results_graded['all'], reranked_results_binary['all'])

Reranking Results: {'bpref': 0.3752, 'ndcg_cut_10': 0.4068} {'P_10': 0.236, 'recip_rank': 0.3921}
